In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Wyświetla wszystkie wiersze
pd.set_option('display.max_columns', None)  # Wyświetla wszystkie kolumny
pd.set_option('display.width', 1000)  # Szerszy output, by uniknąć zawijania
pd.options.display.float_format = '{:.2f}'.format

import sys
import os
sys.path.append(os.path.abspath(".."))

from bot.data.historic.binance_client_wrapper import BinanceClientWrapper
from bot.data.historic.binance_data_fetcher import BinanceDataFetcher
from bot.core.utils import Utils

# client = BinanceClientWrapper()
# client.get_klines('BTCUSDC', '1h', 1735689600000, 1740614400000)

fetcher = BinanceDataFetcher()
df = fetcher.fetch_klines_dates('BTCUSDT', '5m', '2025-01-01 00:00:00', '2025-01-31 23:59:59', 0) # "%Y-%m-%d %H:%M:%S"
df = Utils.add_stoch_rsi(df)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
print(df);

In [ ]:
import pandas as pd

def find_stoch_rsi_periods(df, threshold=20, min_length=15):
    """
    Znajduje okresy, gdzie stoch_rsi_k pozostaje poniżej threshold przez min_length świec
    i zwraca listę DataFrame'ów obejmujących te okresy.
    
    :param df: DataFrame zawierający kolumnę 'stoch_rsi_k' i 'timestamp'.
    :param threshold: Próg dla stoch_rsi_k (domyślnie 20).
    :param min_length: Minimalna liczba świec poniżej threshold (domyślnie 20).
    :return: Lista DataFrame'ów spełniających warunek.
    """
    below_threshold = df['stoch_rsi_k'] < threshold
    periods = []
    start_idx = None

    for i in range(len(df)):
        if below_threshold.iloc[i]:
            if start_idx is None:
                start_idx = i  # Początek serii świec poniżej threshold
        else:
            if start_idx is not None and (i - start_idx) >= min_length:
                # Tworzymy DataFrame dla tego okresu i dodajemy do listy
                periods.append(df.iloc[start_idx:i].copy())
            start_idx = None

    return periods

# Wywołanie funkcji na DataFrame 'df'
stoch_rsi_dfs = find_stoch_rsi_periods(df)

# Wyświetlenie wyników
for idx, df_period in enumerate(stoch_rsi_dfs):
    print(f"Okres {idx + 1}:")
    print(df_period)
    print("-" * 40)
